### Prelude:

In [17]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

In [18]:
data = pd.read_csv("./../Datasets/cleaned_data.csv")

In [20]:
#from Tiffanie's code
# distribution of subjects across treatment conditions (like Table 1 from paper)
# N = 186
treatment_freq = data[["treatment_value", "treatment_frame"]].value_counts()
treatment_rel_freq = data["treatment_frame"].value_counts(normalize=True)
treatment_freq.to_frame().sort_index().join(treatment_rel_freq)

,,count,proportion
treatment_value,treatment_frame,,
0,No framing,22,0.118280
1,Positive science,22,0.118280
2,Negative science,18,0.096774
3,Religious,35,0.188172
4,Equity,30,0.161290
5,Efficiency,32,0.172043
6,Secular,27,0.145161


In [21]:
#Adapted from Tiffanie's code
pd.pivot_table(data, values=['mean_climate_support_before', 'mean_climate_support_after', 'mean_climate_support_change'],
               index=['treatment_value','treatment_frame'],
               aggfunc=['mean'])

mean  \
                                 mean_climate_support_after   
treatment_value treatment_frame                               
0               No framing                         2.012500   
1               Positive science                   2.125000   
2               Negative science                   2.416667   
3               Religious                          2.038095   
4               Equity                             2.080556   
5               Efficiency                         2.213889   
6               Secular                            2.126543   

                                                              \
                                 mean_climate_support_before   
treatment_value treatment_frame                                
0               No framing                          2.016667   
1               Positive science                    1.975000   
2               Negative science                    2.283333   
3               Religious                           1.997619   
4               Equity                              2.022222   
5               Efficiency                          2.134409   
6               Secular                             2.126543   

                                                              
                                 mean_climate_support_change  
treatment_value treatment_frame                               
0               No framing                     -4.166667e-03  
1               Positive science                1.500000e-01  
2               Negative science                1.333333e-01  
3               Religious                       4.047619e-02  
4               Equity                          5.833333e-02  
5               Efficiency                      7.500000e-02  
6               Secular                        -2.055969e-18

In [22]:
pd.pivot_table(data, values=["mean_climate_support_after"],
               index=["party_id", "treatment_value", "treatment_frame"], aggfunc=['mean', 'sem'])

mean  \
                                               mean_climate_support_after   
party_id      treatment_value treatment_frame                               
Democrat      0               No framing                         1.991667   
              1               Positive science                   2.343750   
              2               Negative science                   2.625000   
              3               Religious                          2.131579   
              4               Equity                             2.130952   
              5               Efficiency                         2.388889   
              6               Secular                            2.057692   
Independent   0               No framing                         2.148148   
              1               Positive science                   1.964286   
              2               Negative science                   2.062500   
              3               Religious                          1.916667   
              4               Equity                             1.983333   
              5               Efficiency                         2.089286   
              6               Secular                            2.250000   
No preference 0               No framing                         1.000000   
              1               Positive science                   1.500000   
              3               Religious                          2.166667   
              4               Equity                             2.333333   
              5               Efficiency                         1.750000   
              6               Secular                            2.500000   
Republican    1               Positive science                   2.333333   
              2               Negative science                   1.750000   
              3               Religious                          1.770833   
              4               Equity                             1.958333   
              5               Efficiency                         2.250000   
              6               Secular                            1.805556   

                                                                      sem  
                                               mean_climate_support_after  
party_id      treatment_value treatment_frame                              
Democrat      0               No framing                         0.198548  
              1               Positive science                   0.104982  
              2               Negative science                   0.107044  
              3               Religious                          0.118078  
              4               Equity                             0.132401  
              5               Efficiency                         0.119330  
              6               Secular                            0.198010  
Independent   0               No framing                         0.183090  
              1               Positive science                   0.138321  
              2               Negative science                   0.328744  
              3               Religious                          0.197398  
              4               Equity                             0.278139  
              5               Efficiency                         0.135182  
              6               Secular                            0.163748  
No preference 0               No framing                              NaN  
              1               Positive science                   0.500000  
              3               Religious                          0.416667  
              4               Equity                             0.333333  
              5               Efficiency                              NaN  
              6               Secular                            0.000000  
Republican    1               Positive science                   0.546453  
           